# LAB 1 - A* SEARCH

### Import
---

In [1]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np
from math import ceil
from tqdm.auto import tqdm
import random


### Problem specification
---

In [2]:
class Instance():
    State = namedtuple('State', ['taken', 'not_taken'])

    def __init__(self, PROBLEM_SIZE = 20, NUM_SETS = 40, DENSITY = 0.3):
        self.PROBLEM_SIZE = PROBLEM_SIZE
        self.NUM_SETS = NUM_SETS
        self.DENSITY = DENSITY
        #self.SETS = tuple(np.array([random.random() < self.DENSITY for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
        #self.SETS = tuple(np.random.random(self.PROBLEM_SIZE) < self.DENSITY for _ in range(NUM_SETS))
        self.set_generator()

    def set_generator(self):
        sets = []
        for seed in range(self.NUM_SETS):
            np.random.seed(seed)
            sets.append(np.random.random(self.PROBLEM_SIZE) < self.DENSITY)
        
        self.SETS = tuple(sets)
    def re_seed(self, mul = 1):
        sets = []
        for seed in range(self.NUM_SETS):
            np.random.seed(mul * seed)
            sets.append(np.random.random(self.PROBLEM_SIZE) < self.DENSITY)
        
        self.SETS = tuple(sets)

    def covered(self, state):
        return reduce(np.logical_or, [self.SETS[i] for i in state.taken], np.array([False for _ in range(self.PROBLEM_SIZE)]))

    def goal_check(self, state):
        boolean_val =  np.all(self.covered(state))
        assert not (boolean_val is False and len(state.not_taken) == 0), "Problem not solvable"

        return boolean_val
    
    def __str__(self) -> str:
        return f'size:{self.PROBLEM_SIZE}_numset:{self.NUM_SETS}_density:{self.DENSITY}'
    

class A_star():
    def __init__(self):
        pass

    def Search(self, instance, chosen_key):
        frontier = PriorityQueue()
        init = Instance.State(set(), set(range(instance.NUM_SETS)))
        frontier.put((chosen_key.f(init), init))

        counter = 0
        k, current_state = frontier.get()
        while not instance.goal_check(current_state):
            counter += 1
            for action in current_state[1]:
                new_state = Instance.State(current_state.taken ^ {action}, current_state.not_taken - {action})
                frontier.put((chosen_key.f(new_state),new_state))
            k, current_state = frontier.get()

        
        return counter, current_state

class key():
    def __init__(self, g, h):
        self.g = g
        self.h = h

    def f(self, state):
        return self.g(state) + self.h(state)
    
    

### Functions
---

#### H function

- this is professor's h functions

In [3]:
def h1(instance, state):
    largest_set_size = max(sum(s) for s in instance.SETS)
    missing_size = instance.PROBLEM_SIZE - sum(instance.covered(state))
    optimistic_estimate = ceil(missing_size/largest_set_size)
    return optimistic_estimate

def h2(instance, state):
    already_covered = instance.covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in instance.SETS)
    missing_size = instance.PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size/largest_set_size)
    return optimistic_estimate

def h3(instance, state):
    already_covered = instance.covered(state)
    if np.all(already_covered):
        return 0
    missing_size = instance.PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in instance.SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

- this is my h function

In [4]:
def h4(instance, state):
    return max(h2(instance, state), h3(instance, state))

def distance(instance, state):
    return  instance.PROBLEM_SIZE - sum(instance.covered(state))


### Experiments
---

#### Default instance (small) + h1


In [5]:
default_instance = Instance()

g = lambda state: len(state.taken)
h = lambda s: h1(default_instance,s)
k = key(g,h)

solver = A_star()
number_of_step , result = solver.Search(default_instance, k)

print(default_instance)
print('heuristic function: h1')
print(f'step: {number_of_step}')
print(result)

size:20_numset:40_density:0.3
heuristic function: h1
step: 101
State(taken={36, 21, 15}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39})


#### Default instance (small) + h2


In [6]:
default_instance = Instance()

g = lambda state: len(state.taken)
h = lambda s: h2(default_instance,s)
k = key(g,h)

solver = A_star()
number_of_step , result = solver.Search(default_instance, k)

print(default_instance)
print('heuristic function: h2')
print(f'step: {number_of_step}')
print(result)

size:20_numset:40_density:0.3
heuristic function: h2
step: 14
State(taken={2, 35, 31}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 36, 37, 38, 39})


#### Default instance (small) + h3


In [7]:
default_instance = Instance()

g = lambda state: len(state.taken)
h = lambda s: h3(default_instance,s)
k = key(g,h)

solver = A_star()
number_of_step , result = solver.Search(default_instance, k)

print(default_instance)
print('heuristic function: h3')
print(f'step: {number_of_step}')
print(result)

size:20_numset:40_density:0.3
heuristic function: h3
step: 14
State(taken={2, 35, 31}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 36, 37, 38, 39})


#### Default instance (small) + h4


In [8]:
default_instance = Instance()

g = lambda state: len(state.taken)
h = lambda s: h4(default_instance,s)
k = key(g,h)

solver = A_star()
number_of_step , result = solver.Search(default_instance, k)

print(default_instance)
print('heuristic function: h4')
print(f'step: {number_of_step}')
print(result)

size:20_numset:40_density:0.3
heuristic function: h4
step: 14
State(taken={2, 35, 31}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 36, 37, 38, 39})


#### Large instance

In [9]:
large_instance = Instance(40,80)

g = lambda state: len(state.taken)
h = lambda s: h2(large_instance,s)
k = key(g,h)

solver = A_star()
number_of_step , result = solver.Search(large_instance, k)

print(large_instance)
print('heuristic function: h2')
print(f'step: {number_of_step}')
print(result)

size:40_numset:80_density:0.3
heuristic function: h2
step: 427
State(taken={48, 74, 11, 62}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79})


In [10]:
large_instance = Instance(40,80)

g = lambda state: len(state.taken)
h = lambda s: h3(large_instance,s)
k = key(g,h)

solver = A_star()
number_of_step , result = solver.Search(large_instance, k)

print(large_instance)
print('heuristic function: h3')
print(f'step: {number_of_step}')
print(result)

size:40_numset:80_density:0.3
heuristic function: h3
step: 789
State(taken={1, 66, 35, 5}, not_taken={0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79})


In [11]:
large_instance = Instance(40,80)

g = lambda state: len(state.taken)
h = lambda s: h4(large_instance, s)
k = key(g,h)

solver = A_star()
number_of_step , result = solver.Search(large_instance, k)

print(large_instance)
print('heuristic function: distance')
print(f'step: {number_of_step}')
print(result)

size:40_numset:80_density:0.3
heuristic function: distance
step: 789
State(taken={1, 66, 35, 5}, not_taken={0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79})
